### Set Up Packages and Authorize Access to GEE and Google Drive

In [ ]:
import importlib
import subprocess

# The list of libraries required
libraries = ["ee", "itertools", "jenkspy"]

# Automatically import/install the libraries
for i in libraries:
    try:
        library = importlib.import_module(i)
        print(f"Successfully imported {i}")
    except ImportError:
        print(f"{i} is not installed. Attempting to install it...")
        subprocess.check_call(["pip", "install", i])

        try:
            library = importlib.import_module(i)
            print(f"Successfully imported {i}")
        except ImportError:
            print(f"Failed to install and import {i}. Please install it manually.")

# Initialize GEE's Python API
import ee
ee.Authenticate()
ee.Initialize()

# Authorize access to Google Drive (for exporting data)
from google.colab import drive
drive.mount('/content/drive')

Successfully imported ee
Successfully imported itertools
jenkspy is not installed. Attempting to install it...
Successfully imported jenkspy
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=lgu72LFLsj_iIm6MjFe9Zt2betvHamoRyImJXCeXuOU&tc=ceXMcK1fJVgg-BHz-gy4QbVvLq4o-as_5wjSXcSSjYc&cc=obFknfnn0KVTYi1mPmeJz2gIckZt1U5R4ODZ6gjyb2o

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXno4n2lCTFo7rTv1i83q4cURVG0XT0aUPovWLg70x7k6GC95suQvTQ

Successfully saved authorization token.
Mounted at /content/drive


### Specify the GRES ID of the Reservoir(s) to be processed

In [ ]:
# Enter the list of id from the GRES_Polys that you want to process, it can take a single or multiple ids
GRES_ID = [127,34]

# Where to export the raster on Google Drive?
RasterFolder = "GEE_Raster"

# Where to export the water polygons on Google Drive?
PolyFolder = "GEE_Polygon"

# Where is your polygon data source?
GRes_polys = ee.FeatureCollection('projects/ee-climatechangehydrology1/assets/GRes_Polys')

# What is path of your GEE Asset?
Asset_Folder_Path= 'projects/ee-climatechangehydrology/assets/GRes_ProbabilityRaster/'

### Batch Process Multiple Reservoirs
*   This section of the module uses a "for" loop that iteratively select each id in the list, use it as a string to query data and submit tasks on the server;
Unlike using the .map() function, this iteration only applied to the python
list locally to allow the functions to be used repeatly. This avoid
encountering errors such as "User Memory Exceed" or "Computation Timeout"
due to the GEE's "Per-request Memory Footprint", which is that limited RAM
is assigned for each user for in-memeory data processing.
*   For more info about GEE's computation limits,
    see: https://developers.google.com/earth-engine/guides/usage

*   A preview on some of the aggregation-based parameters:

  "tileScale"(0-16): setting a larger tilescale value means the image is dividied
  into smaller chunks/tiles for parallel processing, this will slow down the
  computation because there are more tiles need to be aggregated at the end.
  However, it will allow the operations to succeed since a large task is being
  splited into many smaller tasks. For small reservoirs, consider adjusting
  the tileScale to 8 or 10 to speed up processing.

  "bestEffort = True": scale/resolution of the image may be reduced to allows
  the computation to succeed at the given value of maxPixels; It's necessary for
  large reservoir, but the result of this may excludes some water edges.
  (similar to using "simplify polygon" in ArcGIS Pro)

  "eightConnected = True": it means a pixel at the center of the 3x3 grid is
  considered connected to all eight surrounding pixels, even the diagonals.
  If false, it uses a four-connected system, only those pixels with four-
  connected neighbourds (top, bottom, left, right) are converted into a polygon.


In [ ]:
# re-state the import statements
# These libraries are needed for "client-side" operations
import jenkspy
from itertools import repeat

for id in GRES_ID:
  try:
    # Get the pre-processed probability raster
    ProbabilityRaster = ee.Image(Asset_Folder_Path + 'GRES_' + str(id))
    # Get some attribute data from "GRES_Polys" to be copied to the results
    feature = GRes_polys.filter(ee.Filter.eq('GRES_ID', id))
    # Get name of the reservoir
    res_name = feature.aggregate_array("DAM_NAME").getString(0).getInfo()
    # Get id of the reservoir in the "GRAND" Database
    grand_id = feature.aggregate_array("GRAND_ID").getNumber(0).getInfo()
    # Get id of the reservoir in the "GeoDAR v1.3" database
    geodar_id = feature.aggregate_array("ID_GRDv13").getNumber(0).getInfo()
    # Get province name ("administration unit")
    province = feature.aggregate_array("ADMIN_UNIT").getString(0).getInfo()
    # Normalize the range of probability values into a 0-1 scale
    Image_MinMax = ProbabilityRaster.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry= ProbabilityRaster.geometry(),
        scale=10,
        maxPixels= 1e13,
        tileScale = 16
    )
    min = Image_MinMax.getNumber('probability_min')
    max = Image_MinMax.getNumber('probability_max')
    norm_prob_ras = ProbabilityRaster.select('probability').unitScale(min,max)

    """
    *** The following section performs a data classifcation ***
    1. Compute a Histogram to store raster statistics in a list, with
        ee.Reducer.fixedHistogram (fixed width, 100 buckets, 0.01 width),
        which is a server-side operation.

        Direct conversion from ee.Image to ee.Array is not possible at this
        given scale and amount of operations; However, we already computed
        a min/max stretched ee.Image and we know that min = 0 and max =1.

    2. Create a list of synthetic data values derived from the histogram to
    replicate the original data distribution of the raster.

    3. Classify the synthetic data into three classes based on their
      "natural breaks" using the "Fisher-Jenks Algorithm" from jenkspy.
      (This is a client-side operation that consumes local RAM, the script
      may takes longer than usual at this step, especially for large
      reservoirs).

    3. Categorize the synthetic data into three classes using the
    "Fisher-Jenks Algorithm" from jenkspy, based on their optimal
    "natural breaks."

      This operation would be processed on the client side using Google Collab's
      VRAM as a way to avoid overloading the gee server; It can take longer than
      usual for extremely large reservoirs.

    """
    ## ee.Reducer.fixedHistogram is a server-side operation;
    ## To avoid computation timeout, we can set maxPixels = 1e13 and
    ## tileScale = 16.
    bucket_count = ee.Array(norm_prob_ras.reduceRegion(
        reducer=ee.Reducer.fixedHistogram(min=0, max=1, steps=100),
        geometry=ProbabilityRaster.geometry(),
        scale=10,
        maxPixels=1e13,
        tileScale=16
    ).get('probability'))


    # The output is a 1x2 array;
    # Reshape the array into a 1x1 array, and then convert it into a ee.List;
    # The ee.list would contain 200 elements in the following order
    # (bucket, count, bucket, count, ....)
    bucket_count = bucket_count.reshape([-1]).toList().flatten()

    # Slice and separate the values into two lists: bucket, count;
    # For bucket, we need to round the values to the nearest two decimals;
    # For count, we need to round the values into the nearest whole number;
    # Use .getInfo() to retrieve the data from the server as python lists;
    # The python list would allow us to use normal python packages;

    bucket = bucket_count.slice(0,201,2)
    bucket = bucket.map(
        lambda number: ee.Number(number).multiply(100).round().divide(100)
    )
    bucket = bucket.getInfo()

    count = bucket_count.slice(1,201,2)
    count = count.map(lambda number: ee.Number(number).round())
    count = count.getInfo()


    # Before we generate a list of synthetic data points,
    # we can avoid unnecessary computation by reducing the the number of
    # repeated values porportionally.The break values won't be affected in
    # this scenario.
    min_count = sorted(count)[0]
    if min_count >100:
      count = [round(x / 100) for x in count]
    elif min_count >10:
      count = [round(x / 10) for x in count]
    else:
      pass

    # After checking for redundant values,
    # we can generate a list of synthetic data points and do a 3-class
    # natural breaks classifcation to identify
    # [non-water, seasonal water, permanent water].
    bucket_count = list(zip(bucket,count))
    syn_data = [bucket for bucket, count in bucket_count for i in repeat(None, int(count))]
    natural_breaks = jenkspy.jenks_breaks(syn_data, n_classes=3)

    # example: natural_breaks = [0.0, 0.33, 0.66, 1.0 ]
    # index 0 to 1 is non-water, 1-2 is seasonal water, 2-3 is permanent water
    max_water = norm_prob_ras.gt(natural_breaks[1]).selfMask()
    min_water = norm_prob_ras.gt(natural_breaks[2]).selfMask()

    # Convert the new layer into vectors
    conversion_params = {
        'geometryType': 'polygon',
        'geometry': ProbabilityRaster.geometry(),
        'scale': 10,
        'eightConnected': True,
        'labelProperty': 'GRES_ID',
        'bestEffort': True,
        'tileScale': 16
    }

    def set_properties(feature):
      return feature.set({
            'GRES_ID': int(id),
            'NAME': str(res_name),
            'BREAK': str(natural_breaks),
            'GRAND_ID': int(grand_id),
            'ID_GRDv13': int(geodar_id),
            'PROV': str(province)
      })

    # The results are multi-part polygons,
    # with a default projection EPSG:4326.
    max_water_poly = max_water.reduceToVectors(**conversion_params)
    min_water_poly = min_water.reduceToVectors(**conversion_params)

    max_water_poly = max_water_poly.map(set_properties)
    min_water_poly = min_water_poly.map(set_properties)

    # Remove those vectors with pixel counts <=1
    max_water_poly = max_water_poly.filter("count != 1").filter("count != 0")
    min_water_poly = min_water_poly.filter("count != 1").filter("count != 0")

    # Set Export Tasks
    Export_ProbailityRaster = ee.batch.Export.image.toDrive(
        image = norm_prob_ras,
        region = norm_prob_ras.geometry(),
        folder = RasterFolder,
        description = 'GRES_' + str(id) + '_GeoTIFF',
        fileFormat = 'GeoTIFF',
        scale = 10,
        maxPixels = 1e13
    )

    Export_MaxWaterPolys = ee.batch.Export.table.toDrive(
        collection=max_water_poly,
        folder=PolyFolder,
        description='GRES_' + str(id) + '_MaxWater',
        fileFormat='SHP'
    )

    Export_MinWaterPolys = ee.batch.Export.table.toDrive(
        collection=min_water_poly,
        folder=PolyFolder,
        description='GRES_' + str(id) + '_MinWater',
        fileFormat='SHP'
    )

    # Initiate the export tasks;
    # Comment out the task if not required;
    Export_ProbailityRaster.start()
    Export_MaxWaterPolys.start()
    Export_MinWaterPolys.start()
    print(f"Operation Successful on GRES_ID: {id}.")

  except Exception as e:
    print(f"Operation Failed on GRES_ID: {id}. Please check your input.")

# Print when execution completes
print("Operation is completed. Check your export tasks on GEE's Task Manager")

Operation Successful on GRES_ID: 127.
Operation Successful on GRES_ID: 34.
Operation is completed. Check your export tasks on GEE's Task Manager
